Importance of Backpropagration:
 - We know how much error each of the hidden units contributed to the final output when updating gradient descent
 - The error resulting from hidden units is scaled by the weights going forward through the network.
 - Since we know the error at the output, we can use the weights to work backwards to hidden layers.

The vanishing gradient problem
- Later addressed (probably as LSTM)

In [1]:
import numpy as np

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1/(1+np.exp(-x))

def forward_pass(x, weights_input_to_hidden, weights_hidden_to_output):
    """
    Make a forward pass through the network
    """
    # Calculate the input to the hidden layer.
    hidden_layer_in = np.dot(x, weights_input_to_hidden)
    # Calculate the hidden layer output.
    hidden_layer_out = sigmoid(hidden_layer_in)

    # Calculate the input to the output layer.
    output_layer_in = np.dot(hidden_layer_out, weights_hidden_to_output)
    # Calculate the output of the network.
    output_layer_out = sigmoid(output_layer_in)
    return hidden_layer_out, output_layer_out

def backward_pass(x, target, learnrate, hidden_layer_out, \
                  output_layer_out, weights_hidden_to_output):
    """
    Make a backward pass through the network
    """
    # TODO: Calculate output error
    # Replace None with appropriate code
    error = target - output_layer_out
    
    # TODO: Calculate error term for output layer
    # Replace None with appropriate code
    output_error_term = error * output_layer_out * (1 - output_layer_out)

    # TODO: Calculate error term for hidden layer
    # Replace None with appropriate code
    hidden_error_term = output_error_term * weights_hidden_to_output * hidden_layer_out * (1 - hidden_layer_out)


    # TODO: Calculate change in weights for hidden layer to output layer
    # Replace None with appropriate code
    delta_w_h_o = learnrate * output_error_term * hidden_layer_out

    
    # TODO: Calculate change in weights for input layer to hidden layer
    # Replace None with appropriate code
    delta_w_i_h = learnrate * hidden_error_term * x[:, None]
    
    return delta_w_h_o, delta_w_i_h

# Create data to run through the network
x = np.array([0.5, 0.1, -0.2])
target = 0.6
learnrate = 0.5
weights_input_to_hidden = np.array([
    [0.5, -0.6],
    [0.1, -0.2],
    [0.1, 0.7]
])
weights_hidden_to_output = np.array([0.1, -0.3])

# Forward pass
hidden_layer_out, output_layer_out = forward_pass(
    x, weights_input_to_hidden, weights_hidden_to_output
)

# Backward pass
delta_w_h_o, delta_w_i_h = backward_pass(
    x, target, learnrate, hidden_layer_out, output_layer_out, \
    weights_hidden_to_output
)

print('Change in weights for hidden layer to output layer:')
print(delta_w_h_o)
print('Change in weights for input layer to hidden layer:')
print(delta_w_i_h)

Change in weights for hidden layer to output layer:
[0.00804047 0.00555918]
Change in weights for input layer to hidden layer:
[[ 1.77005547e-04 -5.11178506e-04]
 [ 3.54011093e-05 -1.02235701e-04]
 [-7.08022187e-05  2.04471402e-04]]


In [ ]:
import numpy as np
from data_prep import features, targets, features_test, targets_test

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

def forward_pass(x, weights_input_to_hidden, weights_hidden_to_output):
    """
    Make a forward pass through the network
    """
    # Calculate the input to the hidden layer.
    hidden_layer_in = np.dot(x, weights_input_to_hidden)
    # Calculate the hidden layer output.
    hidden_layer_out = sigmoid(hidden_layer_in)

    # Calculate the input to the output layer.
    output_layer_in = np.dot(hidden_layer_out, weights_hidden_to_output)
    # Calculate the output of the network.
    output_layer_out = sigmoid(output_layer_in)

    return hidden_layer_out, output_layer_out
    
def backward_pass(x, target, learnrate, hidden_layer_out,
                  output_layer_out, weights_hidden_to_output):
    """
    Make a backward pass through the network
    """
    # Calculate output error
    error = target - output_layer_out

    # Calculate error term for output layer
    output_error_term = error * output_layer_out * (1 - output_layer_out)

    # Calculate error term for hidden layer
    hidden_error_term = np.dot(output_error_term, weights_hidden_to_output) * \
                    hidden_layer_out * (1 - hidden_layer_out)

    # Calculate change in weights for hidden layer to output layer
    delta_w_h_o = learnrate * output_error_term * hidden_layer_out

    # Calculate change in weights for input layer to hidden layer
    delta_w_i_h = learnrate * hidden_error_term * x[:, None]

    return delta_w_h_o, delta_w_i_h
    
def update_weights(weights_input_to_hidden, weights_hidden_to_output, 
                   features, targets, learnrate):
    """
    Complete a single epoch of gradient descent and return updated weights
    """
    delta_w_i_h = np.zeros(weights_input_to_hidden.shape)
    delta_w_h_o = np.zeros(weights_hidden_to_output.shape)

    # Loop through all records, x is the input, y is the target
    for x, y in zip(features.values, targets):
        ## Forward pass ##

        # Calculate the output using the forward_pass function.
        hidden_layer_out, output_layer_out = forward_pass(x,
            weights_input_to_hidden, weights_hidden_to_output)

        ## Backward pass ##

        # Calculate the change in weights using the backward_pass
        # function.
        delta_w_h_o_tmp, delta_w_i_h_tmp = backward_pass(x, y, learnrate,
            hidden_layer_out, output_layer_out, weights_hidden_to_output)
        delta_w_h_o += delta_w_h_o_tmp
        delta_w_i_h += delta_w_i_h_tmp

    n_records = features.shape[0]
    # Update weights  (don't forget division by n_records or number
    # of samples).
    weights_input_to_hidden += delta_w_i_h / n_records
    weights_hidden_to_output += delta_w_h_o / n_records
    
    return weights_input_to_hidden, weights_hidden_to_output

def gradient_descent(features, targets, epochs=2000, learnrate=0.9):
    """
    Perform the complete gradient descent process on a given dataset
    """
    # Use to same seed to make debugging easier
    np.random.seed(11)
    
    # Initialize loss and weights
    last_loss = None
    n_features = features.shape[1]
    n_hidden = 2
    weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
    weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

    # Repeatedly update the weights based on the number of epochs
    for e in range(epochs):
        weights_input_hidden, weights_hidden_output = update_weights(
            weights_input_hidden, weights_hidden_output, features,
            targets, learnrate
        )

        # Printing out the MSE on the training set every 10 epochs.
        # Initially this will print the same loss every time. When all of
        # the TODOs are complete, the MSE should decrease with each
        # printout
        if e % (epochs / 10) == 0:
            hidden_output = sigmoid(np.dot(features, weights_input_hidden))
            out = sigmoid(np.dot(hidden_output,
                                 weights_hidden_output))
        loss = np.mean((out - targets) ** 2)
            if last_loss and last_loss < loss:
                print("Train loss: ", loss, "  WARNING - Loss Increasing")
            else:
                print("Train loss: ", loss)
            last_loss = loss
            
    return weights_input_hidden, weights_hidden_output

def calculate_accuracy(features, targets, weights_input_hidden,
                       weights_hidden_output):
    """
    Given features, targets, and weights for both hidden and output
    layers, calculate the accuracy of predictions
    """
    hidden_out = sigmoid(np.dot(features, weights_input_hidden))
    output_out = sigmoid(np.dot(hidden_out, weights_hidden_output))
    predictions = output_out > 0.5
    accuracy = np.mean(predictions == targets)
    return accuracy

# Calculate accuracy on test data
weights_input_hidden, weights_hidden_output = gradient_descent(
    features, targets)
accuracy = calculate_accuracy(features_test, targets_test,
                              weights_input_hidden, weights_hidden_output)
print("Prediction accuracy: {:.3f}".format(accuracy))